In [2]:
#ALLOW GOOGLE DRIVE ACCESS
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#PARAMETERS TO MODIFY
wavelength = 550 # [=] nm, wavelength of LED shined on sample
devicebandgap = 1.61 # [=] eV
multiplier = 32340.219 # fraction of beam photons that hit the field of view, divided by area of field of view
                            # = 11071398   for 100x objective PL field of view
                            # = 32340.219  for 5x objective PL field of view
                            # = 111111.11  for 100x objective 4-point photoconductivity measurement 
                            # = 32267.333  for 5x objective 4-point photoconductivity measurement
                            # = 1138353.3  for 100x objective 2-point photoconductivity measurement
                            # = 33467.333  for 5x objective 2-point photoconductivity measurement

                        
                        
#RESPONSIVITY
import math
text = open("drive/Team Drives/Hillhouse Group/Equipment & Procedures/Lumencor Calibration/Responsivity.txt").read().split('\n')
line = text[int((wavelength - 200) / 10)].split('\t') # Subtract 200 b/c 200 is the first wavelength measured and divide by 10 b/c of step size. 
resp = float(line[1]) # responsivity [=] Amps/Watt



#PHOTON ENERGY
h = 6.62607004e-34 # [=] m^2 kg / s
c = 299792458 # [=] m/s
E = (h * c) / (wavelength * (1e-9)) # [=] J/photon
q = 1.6021766208e-19 # [=] J/eV



#AM1.5 INTEGRATION
import numpy as np
from scipy.integrate import trapz
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

device_bandgap = (h * c * 1e9)/(devicebandgap * q) # [=] nm

am15_nm = np.loadtxt('drive/Team Drives/Hillhouse Group/Equipment & Procedures/Lumencor Calibration/am15_nmdata.txt', delimiter = ',')
am15_energy_pernm = np.loadtxt('drive/Team Drives/Hillhouse Group/Equipment & Procedures/Lumencor Calibration/am15_spec_data.txt', delimiter = ',')
#Calculate above bandgap photon flux (AM 1.5) for test cell
am15_photon_pernm = (am15_energy_pernm/q)*am15_nm/(1240)
#plt.plot(am15_nm,am15_photon_pernm)
am15_idx = np.argmin(abs(am15_nm-device_bandgap))
above_bandgap_photon_flux = trapz(am15_photon_pernm[:am15_idx],am15_nm[:am15_idx]) #photons/m^2-sec

sq_jsc = above_bandgap_photon_flux*q*(1/100)**2*1000 #mA/cm^2
#print("device bandgap = " + str(device_bandgap) + " nm")
print("AM1.5 above bandgap photon flux = " + str(above_bandgap_photon_flux) + " photons/(m^2*s)")
#print("Shockley Queisser Jsc = " + str(sq_jsc) + " mA/cm^2")



#CONVERSION FROM 1 SUN INTO A POWER METER READING
powermeter = above_bandgap_photon_flux * E * resp / multiplier # [=] A
print("At 1 sun, the power meter should read " + str(powermeter) + " Amps")


AM1.5 above bandgap photon flux = 1.5625251085036185e+21 photons/(m^2*s)
At 1 sun, the power meter should read 0.004968308143391814 Amps
